# Interface Demonstration Notebook

A notebook to demonstrate an interface with some of the features of the Probability of Failure Model

Author: gavin.treseder@essentialenergy.com.au

In [5]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from config import config
from pof.loader.asset_model_loader import AssetModelLoader
from pof import Component, FailureMode, Task
from pof.interface.dashlogger import DashLogger
from pof.interface.layouts import *
from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig

In [6]:
# Quick test to make sure everything is works 
comp = Component.demo()
mcl = make_component_layout(comp)
print(f"Validation Result: {validate_layout(comp, mcl)}")

Validation Result: True


In [7]:
# Turn off logging
logging.getLogger().setLevel(logging.INFO)
config['Load']['on_error_use_default'] ='False'

filename = os.path.dirname(os.getcwd()) + r"\data\inputs\Asset Model - Pole - Timber.xlsx"

aml = AssetModelLoader(filename)
comp_data = aml.load()
comp = Component.from_dict(comp_data['pole'])

In [8]:
comp.mc_timeline(200)

100%|██████████| 100/100 [00:06<00:00, 14.50it/s]


In [9]:
%%timeit
comp.expected_risk_cost_df()


171 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
comp.expected_risk_cost_df_legacy_method()

KeyError: "['source', 'active'] not in index"

In [79]:

d_fm = {}
for fm in comp.fm.values():
    d_task = {}
    for task in fm.tasks.values():
        d_task[task.name] = pd.DataFrame(task.expected_costs())
    df_task = pd.concat(d_task, names=['source', 'drop']).reset_index().drop('drop', axis=1)
    d_fm[fm_name] = df_task

df_fm = pd.concat(d_fm, names=['failure_mode', 'drop']).reset_index().drop('drop', axis=1)


In [86]:
df_fm._exp

,failure_mode,source,time,cost,active
0,weathering,inspection,100.0,0.0,True
1,weathering,CAT1/2 replacement,29.0,7000.0,True
2,weathering,CAT1/2 replacement,30.0,7000.0,True
3,weathering,CAT1/2 replacement,31.0,7000.0,True
4,weathering,CAT1/2 replacement,33.0,7000.0,True
5,weathering,CAT1/2 replacement,34.0,14000.0,True
6,weathering,CAT1/2 replacement,35.0,7000.0,True
7,weathering,CAT1/2 replacement,36.0,7000.0,True
8,weathering,CAT1/2 replacement,39.0,7000.0,True
9,weathering,CAT1/2 replacement,40.0,7000.0,True


In [138]:
df = pd.DataFrame(comp.fm['termites']._expected_risk_cost()).T.apply(fill_blanks, axis=1, args=(0, 200)).apply(fill_blanks, axis=1, args=(0, 200))
df.index.name = 'task'
df_cost = df.explode('cost')['cost']
df = df.explode('time')
df['cost'] = df_cost

In [4]:
layout = html.P(id='ffcf')

In [12]:

# Turn off logging level to speed up implementation
logging.getLogger().setLevel(logging.CRITICAL)
logging.getLogger("werkzeug").setLevel(logging.WARNING)

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

pof_sim = copy.copy(comp)
sens_sim = copy.copy(comp)

def layout():
    mcl = make_component_layout(comp)
    update_list = [{"label": option, "value": option} for option in comp.get_update_ids()]

    layout = html.Div([
        html.Div(id='log'),
        html.Div(children='Update State:', id='update_state'),
        html.Div([
            html.P(children='Sim State', id='sim_state'),
            html.P(id='sim_state_err', style={'color': 'red'}),
        ]),
        html.Div(children='Fig State', id='fig_state'),
        html.Div([
            dcc.Interval(id="progress-interval", n_intervals=0, interval=50),
            dbc.Row([
                dbc.Progress(id="progress"),
                dcc.Input(id='n_iterations-input', value=10, type='number'),
            )]
            dbc.Progress(id="progress_sens"),
        ]),
        dbc.Row([dbc.Col(), dbc.Col(dcc.Dropdown(
            id='demo-dropdown',
            options=update_list,
            value='comp-fm-slow_aging-tasks-inspection-t_interval'))]),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='pof-fig')),
                dbc.Col(dcc.Graph(id='insp_interval-fig')),
            ]
        ),
            dbc.Row(
            [
                dbc.Col(dcc.Graph(id='cond-fig')),
                dbc.Col(dcc.Graph(id="cost-fig"))
            ]
        ),
        mcl,
    ])
    
    return layout

# Layout
var_to_scale = cf.scaling
app.layout = layout

collapse_ids = comp.get_objects()

@app.callback(
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapse_ids],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapse_ids: #TODO change to is not None

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return is_open


ms_fig_update = comp.get_dash_ids()
param_inputs = [Input(dash_id,"checked") if 'active' in dash_id else Input(dash_id,"value") for dash_id in ms_fig_update]

# Update --> Simulate --> Figures

@app.callback(
    Output('update_state', 'children'), 
    param_inputs
)
def update_parameter(graph_y_limit_active, graph_y_limit, *args):
    """Update a the pof object whenever an input is changes"""

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = None
    value=None  

    # If any parameters have changed update the objecte
    if ctx.triggered:
        dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
        value = ctx.triggered[0]['value']

        # Scale the value if req
        value = value / var_to_scale.get(dash_id.split('-')[-1], 1)

        # update the model
        comp.update(dash_id, value)
    
    return f"Update State: {dash_id} - {value}"


@app.callback(
    [Output("sim_state", 'children'), Output("sim_state_err", "children")],
    [Input("n_iterations-input", "value"), Input("update_state", "children")],
)
def update_simulation(n_iterations, *args):
    """ Triger a simulation whenever an update is completed or the number of iterations change"""
    global pof_sim
    global sim_err_count

    pof_sim.cancel_sim()
    #time.sleep(1)
    pof_sim = copy.copy(comp)

    pof_sim.next_sim(t_end=200, n_iterations=n_iterations)

    if not pof_sim.up_to_date:
        sim_err_count = sim_err_count + 1
        return dash.no_update, f"Errors: {sim_err_count}"
    return f"Sim State: {pof_sim.n_iterations} - {n_iterations}", ""

fig_start = 0
fig_end = 0

# After a simulation the following callbacks are triggered

@app.callback(
    [Output("cost-fig", "figure"),
    Output("pof-fig", "figure"),
    Output("cond-fig", "figure"),
    Output("fig_state", "children")], 
    [Input("sim_state", "children")]
)
def update_figures(*args):
    global fig_start
    global fig_end

    fig_start = fig_start + 1
    cost_fig = update_cost_fig(pof_sim) #legend = dropdown value
    pof_fig = update_pof_fig(pof_sim)
    cond_fig = update_condition_fig(pof_sim)
    fig_end = fig_end + 1
    return cost_fig, pof_fig, cond_fig, f"Fig State: {fig_start} - {fig_end}"

@app.callback(
    Output("ffcf", "children"), 
    [Input("sim_state", "children")]
)
def update_ffcf(*args):
    cf = pof_sim.expected_cf()
    ff = pof_sim.expected_ff()

    ratio = round(ff / (cf+ff), 2)

    return f"Conditional {cf} : {ff} Functional. {ratio}%"

@app.callback(
    Output("insp_interval-fig", "figure"), 
    [Input('cost-fig', 'figure')]
)
def update_insp_interval(*args):
    """ Trigger a sensitivity analysis of the target variable"""
    # Copy from the main model
    sens_sim = copy.deepcopy(comp)
    
    insp_interval_fig = make_inspection_interval_fig(sens_sim, t_min=1, t_max=10, step=1, n_iterations = 100)

    return insp_interval_fig

# The following progress bars are always running

@app.callback(
    [Output("progress", "value"), Output("progress", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress(n):
    if pof_sim.n is None:
        raise Exception("no process started")
    progress = int(pof_sim.n / pof_sim.n_iterations * 100)

    return progress, f"{progress} %" if progress >= 5 else ""

@app.callback(
    [Output("progress_sens", "value"), Output("progress_sens", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress_sens(n):
    if sens_sim.n is None:
        raise Exception("no process started")
    progress = int(sens_sim.sens_progress() * 100)

    return progress, f"{progress} %" if progress >= 5 else ""


app.run_server(debug=False, mode='inline')
#app.run_server(debug=True, use_reloader=False)

In [6]:
comp.fm['termites']._t_failures#timeline#.tasks['inspection'].sim_timeline(200)

[]

In [7]:
erc = dict()
for fm_name, fm in comp.fm.items():
    erc[fm_name] = fm._expected_risk_cost()

,risk,inspection,CAT3/4 replacement,CAT1/2 replacement
time,"[34, 39, 67]",[],[],"[34, 39, 67]"
cost,"[50000.0, 50000.0, 50000.0]",[],[],"[7000.0, 7000.0, 7000.0]"
active,True,NaN,NaN,NaN
task_active,NaN,True,True,True


In [22]:
list(comp.fm['weathering'].expected_risk_cost()['risk'])

['time', 'cost']

In [16]:
df = comp.expected_risk_cost_df().groupby(by=['failure_mode', 'task']).max()
df

time   cost task_active  \
failure_mode            task                                          
fire_damage             CAT1/2 replacement    61      0        True   
                        inspection            61      0        True   
                        risk                  61      0         NaN   
fungal decay _ external CAT1/2 replacement    62    777        True   
                        CAT3/4 replacement    62      0        True   
                        inspection            62     50        True   
                        risk                  62   5555         NaN   
fungal decay _ internal CAT1/2 replacement    62    777        True   
                        CAT3/4 replacement    62      0        True   
                        inspection            62     50        True   
                        pole_saver_rod        62      0        True   
                        risk                  62   5555         NaN   
impact                  CAT1/2 replacement    61      0        True   
                        inspection            61      0        True   
                        risk                  61      0         NaN   
lightning               CAT1/2 replacement    61      0        True   
                        inspection_1          61      0        True   
                        inspection_2          61     50        True   
                        risk                  61      0         NaN   
termites                CAT1/2 replacement    62    777        True   
                        CAT3/4 replacement    62      0        True   
                        inspection            62     50        True   
                        risk                  62   5555         NaN   
                        termite_powder        62      0        True   
weathering              CAT1/2 replacement    55   1555        True   
                        CAT3/4 replacement    55      0        True   
                        inspection            55      0        True   
                        risk                  55  11111         NaN   

                                            cost_cumulative  fm_active  
failure_mode            task                                            
fire_damage             CAT1/2 replacement                0       True  
                        inspection                        0       True  
                        risk                              0       True  
fungal decay _ external CAT1/2 replacement             1554       True  
                        CAT3/4 replacement                0       True  
                        inspection                      409       True  
                        risk                          11110       True  
fungal decay _ internal CAT1/2 replacement              777       True  
                        CAT3/4 replacement                0       True  
                        inspection                      403       True  
                        pole_saver_rod                    0       True  
                        risk                           5555       True  
impact                  CAT1/2 replacement                0       True  
                        inspection                        0       True  
                        risk                              0       True  
lightning               CAT1/2 replacement                0       True  
                        inspection_1                      0       True  
                        inspection_2                    398       True  
                        risk                              0       True  
termites                CAT1/2 replacement              777       True  
                        CAT3/4 replacement                0       True  
                        inspection                      414       True  
                        risk                           5555       True  
                        termite_powder                    0       True  
weathering     

In [ ]:
erc = dict()
for fm_name, fm in self.fm.items():
    erc[fm_name] = fm.expected_risk_cost_df()

df = pd.concat(erc)
df = (
    df.reset_index()
    .drop(columns="level_1")
    .rename(columns={"level_0": "failure_mode"})
)

In [14]:
comp.fm['termites'].expected_risk_cost_df()

,task,time,cost,task_active,cost_cumulative,fm_active
0,inspection,0,0,True,0,True
1,inspection,1,0,True,0,True
2,inspection,2,0,True,0,True
3,inspection,3,0,True,0,True
4,inspection,4,0,True,0,True
...,...,...,...,...,...,...
310,risk,58,0,NaN,5555,True
311,risk,59,0,NaN,5555,True
312,risk,60,0,NaN,5555,True
313,risk,61,0,NaN,5555,True


In [13]:
df['time'].max()

62

In [9]:
%%timeit

comp.expected_inspection_interval(t_min=1, t_max=10)

In [7]:
pof_sim.cancel_sim()
pof_sim = copy.copy(comp)
pof_sim.next_sim(t_end=200)

In [26]:
comp.reset()

In [30]:
pof_sim.fm['random'].timeline

{}

In [13]:
task.active

False